In [1]:
import json
from tqdm import tqdm

import numpy as np
import torch
from torch import nn, optim
from torchviz import make_dot

from collections import defaultdict

from torch.utils import data
from trajdata import AgentBatch, AgentType, UnifiedDataset
from trajdata.augmentation import NoiseHistories

from modules.model_registrar import ModelRegistrar
from modules.snce import SocialNCE
# from models.trajectory_predictor import TrajectoryPredictor


In [2]:
with open('../config/pedestrians.json', 'r', encoding="utf-8") as config_json:
    hyperparams = json.load(config_json)
hyperparams["device"] = "cpu"
hyperparams["edge_encoding"] = True
hyperparams["contrastive_weight"] = 50.0
hyperparams["history_sec"] = 2.8
hyperparams["prediction_sec"] = 4.8
hyperparams["incl_robot_node"] = False
hyperparams["map_encoding"] = False
hyperparams["preprocess_workers"] = 16
hyperparams["trajdata_cache_dir"] = "../data/pedestrian_datasets/.unified_data_cache"

hyperparams["log_p_yt_xz_max"] = 6
hyperparams["single_mode_multi_sample"] = False
hyperparams["single_mode_multi_sample_num"] = 50

# hyperparams

In [3]:
desired_data=[
    "eupeds_eth-train",
]
data_dirs = {
    "eupeds_eth": "~/Projects/trajectron/datasets/eth_ucy_peds",
}

attention_radius = defaultdict(
    lambda: 20.0
)  # Default range is 20m unless otherwise specified.
attention_radius[(AgentType.PEDESTRIAN, AgentType.PEDESTRIAN)] = 5.0

input_noise = 0.0
augmentations = list()
if input_noise > 0.0:
    augmentations.append(NoiseHistories(stddev=input_noise))

In [4]:
batch_size = 4

dataset = UnifiedDataset(
    desired_data=desired_data,
    history_sec=(0.1, hyperparams["history_sec"]),
    future_sec=(0.1, hyperparams["prediction_sec"]),
    agent_interaction_distances=attention_radius,
    incl_robot_future=hyperparams["incl_robot_node"],
    incl_raster_map=hyperparams["map_encoding"],
    only_predict=[AgentType.PEDESTRIAN],
    no_types=[AgentType.UNKNOWN],
    augmentations=augmentations if len(augmentations) > 0 else None,
    num_workers=hyperparams["preprocess_workers"],
    cache_location=hyperparams["trajdata_cache_dir"],
    data_dirs=data_dirs,
    verbose=True,
)

dataloader = data.DataLoader(
    dataset,
    collate_fn=dataset.get_collate_fn(pad_format="right"),
    pin_memory=False if hyperparams["device"] == "cpu" else True,
    batch_size=batch_size,
    shuffle=True,
    num_workers=hyperparams["preprocess_workers"],
    sampler=None,
)

batch: AgentBatch = next(iter(dataloader))
enc = torch.rand((batch_size, 64), requires_grad=True)

Loading data for matched scene tags: ['train-zurich-eupeds_eth', 'cyprus-train-eupeds_eth']


Calculating Agent Data (Serially): 100%|██████████| 1/1 [00:00<00:00, 2736.01it/s]

1 scenes in the scene index.



Structuring Agent Data Index: 100%|██████████| 1/1 [00:00<00:00, 2786.91it/s]


In [5]:
# Check for valid agent future
batch.agent_fut_len

tensor([10,  8,  1,  2])

In [6]:
model_dir = "./"
model_registrar = ModelRegistrar(model_dir, hyperparams["device"])

In [7]:
snce = SocialNCE(
    feat_dim=64,
    proj_head_dim=hyperparams["proj_head_dim"],
    event_enc_dim=hyperparams["event_enc_dim"],
    snce_head_dim=hyperparams["snce_head_dim"],
    hyperparams=hyperparams,
    device=hyperparams["device"]
)

In [8]:
loss_snce = snce(enc, batch)

In [9]:
print(f"SNCE Loss: {loss_snce}")
print(f"Loss grad_fn: {loss_snce.grad_fn}")

SNCE Loss: 2.3993022441864014
Loss grad_fn: <NllLossBackward0 object at 0x734e3d6a5520>
